#BERT base

In [ ]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

#load the preprocessed dataset
dataset = load_dataset('csv', data_files={'train': 'df_train.csv',
                                          'val': 'df_valid.csv',
                                              'test': 'df_test.csv'})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2cec45cfe0e5b614/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#import the bert base tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
#preprocess function to tokenize the text
def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length',
                                max_length=128,
                                truncation=True,
                                return_tensors="pt")

In [ ]:
#dictionaries to map ids to labels and vice versa
id2label = {0: "phrase", 1: "passage", 2: "multi"}
label2id = {"phrase": 0, "passage": 1, "multi": 2}

In [ ]:
#import the bert base model and pass it to classification model of huggingface

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased", num_labels=3, id2label=id2label, label2id=label2id
).to("cuda")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
#tokenize the data in train, validation and test sets
tokenized_data = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2560 [00:00<?, ? examples/s]

Map:   0%|          | 0/640 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
#import the evaluation metrics - accuracy and f1
import evaluate
import numpy as np

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):

  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"]
  accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]

  return {"f1": f1, "accuracy": accuracy}

In [ ]:
#import the data collator
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

###BERT-base training



In [ ]:
#defining the training arguments for fine-tuning the bert-base model
training_args = TrainingArguments(
    output_dir="output",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,No log,0.858514,0.562646,0.590625
2,No log,0.814094,0.612080,0.643750
3,No log,0.891824,0.613780,0.645312
4,0.736900,0.980457,0.615846,0.645312
5,0.736900,1.023788,0.610417,0.640625


TrainOutput(global_step=800, training_loss=0.5718225193023682, metrics={'train_runtime': 329.7012, 'train_samples_per_second': 38.823, 'train_steps_per_second': 2.426, 'total_flos': 841962936729600.0, 'train_loss': 0.5718225193023682, 'epoch': 5.0})

###BERT-base inference


In [ ]:
# Load trained model
model_path = "output/checkpoint-800"
model = AutoModelForSequenceClassification.from_pretrained(
    model_path, num_labels=3, id2label=id2label, label2id=label2id
).to("cuda")

# Define test trainer
test_trainer = Trainer(model) 
# Make prediction
raw_pred, labels, metrics = test_trainer.predict(tokenized_data["test"]) 
# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

In [ ]:
#compute f1 score
test_f1 = f1_metric.compute(predictions=y_pred, references=labels, average="macro")["f1"]
test_f1

0.6618399351725451

In [ ]:
#compute accuracy score
test_accuracy = accuracy_metric.compute(predictions=y_pred, references=labels)["accuracy"]
test_accuracy

0.66625